Made by [Julia Ive](mailto:j.ive@imperial.ac.uk), [Zhenhao Li](mailto:zhenhao.li18@imperial.ac.uk), and [Nihir](mailto:nv419@ic.ac.uk).

 [The AI Core](https://theaicore.com)
 
Download GloVe!

# Contents
- [RNNs Recap and Primer](#RNNs-Recap-and-Primer)
 - [RNNs for text classification](#RNNs-for-Text-Classification-Task)
 - [Bi-directional RNNs](#BiDirectional-RNNs)
- [RNNs for Language Modelling](#RNNs-for-Language-Modelling)
 - [Evaluation of Language Models](#Evaluation-of-Language-Models)
 - [Long short term memory architectures LSTMs vs. RNNs](#Long-short-term-memory-architectures-LSTMs-vs.-RNNs)
- [Sequence to sequence model](#Sequence-to-sequence-model)
 - [BLEU Score](#BLEU-Score)

# Recurrent Neural Networks

RNNs are designed to make use of sequential data, when the current step has some kind of relation with the previous steps. This makes them ideal for applications with a time component (audio, time-series data) and natural language. RNNs are networks for which value of a unit depends on its own previous output as input.

![](images/RNN_basic.JPG)


An input vector representing the current input element $x_t$ is multiplied by a weight matrix $W$ and then passed through an activation function to compute an activation value for a layer of hidden units. This hidden layer is, in turn, used to calculate a corresponding output, $y_t$. 

$h_t = g(Uh_{t-1}+Wx_t)$

$y_t = a(Vh_t)$

The hidden layer from the previous time step $h_{t-1}$ provides a form of memory, or context, that encodes earlier processing and informs the decisions to be made at later points in time. $U$ determine how the network should make use of past context. RNNs do not impose any limit on this prior context. The context includes information dating back to the beginning of the sequence. Three sets of weights are updated at each timestep: $W$, $U$ and $V$.


![rnn_cell](images/rnn_cell.png)

## Optimization

Surprisingly, with this increased complexity in structure, the optimization method does not become any more difficult. Despite having a different name, back-propagation through time, it is essentially the same thing. All you do is feed in your sequence sequentially to get the output, as usual. You then just calculate your error at each timestep and sum it as opposed to calculating the error at a single timestep like standard neural networks. Then you can use gradient descent to update your weights iteratively until you are satisfied with your network's performance.

![](images/RNN_BPTT.JPG)


## RNNs for Text Classification Task

There are many variations of RNN’s likely Many-One, Many-Many, etc. In our case we aim to classify the input text into positive and negative class. So, set of Many-One LSTM units achieves the task as only one value needs to be outputted for determining the polarity of the review. Usually this is the last RNN hidden state as the one that summarises the whole sequence.


Q: Why RNNs better than FFNNs?

FFNNs do not take context into account. Each word is represented by its embedding independent of other words. RNNs encodes each new word (token) considering the previous words. Meaning of words can change depending on the context. For example, compare the meanings of the word "mean" in those two sentences "I compute the mean" and "His behaviour was mean".

Q: How the model is evaluated ?

The natural choise is accuracy - 
percentage of all the examples that our model labeled correctly. However, it is not good for unbalanced datasets. Imagine a dataset with 999,900 positive examples and 100 negative examples. A very bad classifier can assign positive class to all the examples. This classifier would have 999,900 true negatives and only 100 false negatives and an accuracy of
999,900/1,000,000 or 99.99%! 

Other metrics, more useful for such datasets are: precision, recall and F-measure. Precision measures the percentage of the items that the system labelled as positive and are positive accroding to the gold labels. Recall measures the percentage of items labelled as positive out of all the gold positive items. F-score is the weighted harmonic mean of the precision and recall. 


We will work with a popular classification task of sentiment analysis, the extraction of the sentiment that a writer expresses toward something he/she describes.

![rnn_classification](images/rnn_classification.png)

In [ ]:
import torch
from torchtext import data
#We will work with a dataset from the torchtext package consists of data processing utilities and popular datasets for NLP
from torchtext import datasets
import random
import torch.nn as nn
import time
import math

# We fix the seeds to get consistent results for each training.

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Helper function to print time between epochs
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# With TorchText Field we define how our data will be processed: here we will use Spacy for tokenisation

TEXT = data.Field(tokenize = 'spacy', lower=True)
LABEL = data.LabelField(dtype = torch.long)

# We will experiment with a widely used Stanford Treebank dataset and will predict sentiment of movie reviews
# Our data will be classified in three labels: positive, negative and neutral
# We take the standard split

train_data, valid_data, test_data = datasets.SST.splits(
            TEXT, LABEL)

# Print stat over the data

print('train.fields:', train_data.fields)
print('len(train):', len(train_data))
print('vars(train[0]):', vars(train_data[0]))

In [ ]:
# Now we build a vocabulary out of tokens available from the pre-trained embedding list and the vocabulary of labels.

TEXT.build_vocab(train_data, vectors="glove.6B.50d")
LABEL.build_vocab(train_data)

print('Text Vocabulary Length', len(TEXT.vocab))
print ("Label Vocabulary Length: ", len(LABEL.vocab))

#We can display the most common words in the vocabulary and their frequencies

print(TEXT.vocab.freqs.most_common(20))

#We can also see the vocabulary directly using the stoi (string to int)

print(LABEL.vocab.stoi)

In [ ]:
BATCH_SIZE = 64

# place the tensors on the GPU if available

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

# BucketIterator is an iterator that will return a batch of examples of similar lengths, minimizing the amount of padding per example.
# Padding refers to fixing the length of inputs (adding a reserved token a certain amount of times to match certain length), usually to the max length within a batch. For exmaple:
# i         like  this  movie <pad>
# the       movie is    very  good
# excellent !     <pad> <pad> <pad>

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
            (train_data, valid_data, test_data), 
            batch_size = BATCH_SIZE,
            sort_within_batch = True,
            device = device)

In [ ]:
print(train_iterator)

for batch in train_iterator:
    demo_batch = batch
    break
    
print(demo_batch)

print()

# Note that demo_batch.text has a shape of [sentence length x batch size]
print("Demo batch `text` shape:", demo_batch.text.shape)
# We can simply reshape this into the more familiar [batch size x sentence length]
print("Demo batch `text` transpose shape:", demo_batch.text.T.shape)
print("Demo batch `text` sample: \n", demo_batch.text.T[:3, :])

print()

print("Demo batch `label` shape:", demo_batch.label.shape)
# shape(demo_batch.label.shape) = [batch size]
print("Demo batch `label` sample:", demo_batch.label[:3])

#### The RNN class

Within the constructor we define the layers:
 - An **embedding layer** which acts as a lookup table to map our tokens to their vector 
 - An **RNN** 
 - A **linear layer**. This layer receives the last hidden state from the RNN and outputs logits of `output dim` dimensionality 

All the parameters initialized to random values by default, unless explicitly specified.


In [ ]:
class RNN(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim,
                 bidirectional, dropout, pad_idx):

        super().__init__()

        self.bidirectional = bidirectional
        self.hidden_dim = hidden_dim
        
        # An embedding layer (look-up layer) transforms word indicies into word embeddings. 
        # Here, we initialize our model with pre-trained embeddings (100D pre-trained GloVe embeddings in our case).
        # This layer will fine-tune these embeddings, specific to this model/dataset.
        self.embedding = nn.Embedding.from_pretrained(TEXT.vocab.vectors)
        # We can also train the embeddings from scratch:
        #self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        # An RNN layer. we specify that the batch dimension goes first
        # We have a bidirectional flag which indicates whether the model is unidirectional or bidirectional
        # RNNs can be stacked - i.e. have multiple layers. Here, we will only look at the 1 layer case.
        self.rnn = nn.RNN(embedding_dim,
                          hidden_dim,
                          batch_first=True,
                          bidirectional=bidirectional,
                          num_layers=1)

        # The linear layer takes the final hidden state and feeds it through a fully connected layer.
          # The dimensionality of the output is equal to the output class count.
          # For classification in a bidirectional RNN we concatenate:
            #  - The last hidden state from the forward RNN (obtained from final word of the sentence)
            #  - The last hidden state from the backward RNN (obtained from the first word of the sentence)
          # Due to the concatenation, our hidden size is doubled.
        
        if self.bidirectional:
            linear_hidden_in = hidden_dim * 2
        else:
            linear_hidden_in = hidden_dim

        # The classification (linear) layer
        self.fc = nn.Linear(linear_hidden_in, output_dim)

        # We apply dropout technique that sets a random set of activations in the max-pooling layer to zero.
          # This prevents the network from learning to rely on specific weights and helps to prevent overfitting. 
          # Note that the dropout layer is only used during training, and not during test time.
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # ACRONYMS:
          # B = Batch size
          # T = Max sentence length
          # E = Embedding dimension
          # D = Hidden dimension
          # O = Output dimension

        # shape(text) = [B, T]

        embedded = self.dropout(self.embedding(text))
        # shape(embedded) = [B, T, E]
        
        # An RNN in PyTorch returns two values:
        # (1) All hidden states of the last RNN layer
        # (2) Hidden state of the last timestep for every layer
          # Note: we are only using 1 layer
        all_hidden, last_hidden = self.rnn(embedded)
        # shape(all_hidden) = [B, T, D*num_directions]
        # shape(last_hidden) = [num_layers*num_directions, B, D].  num_layers = 1
        # NOTE. If we were to NOT use the `batch_first` flag, shape of all_hidden would be [T, B, D*num_directions]
        
        if self.bidirectional:
            # Concat the final forward (hidden[0,:,:]) and backward (hidden[1,:,:]) hidden layers
            last_hidden = torch.cat((last_hidden[0, :, :], last_hidden[1, :, :]), dim=-1)
            # shape(last_hidden) = [B, D*2]

        else:
            last_hidden = last_hidden.squeeze(0)
            # shape(last_hidden) = [B, D]

        # Our predictions.
        logits = self.fc(self.dropout(last_hidden))
        # shape(logits) = [B, O]
        
        return logits



In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 128
OUTPUT_DIM = len(LABEL.vocab)
BIDIRECTIONAL = False
DROPOUT = 0.3
# get our pad token index from the vocabulary
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

Let's initialise the RNN now

In [ ]:
model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
def accuracy(preds, y):
    """
    returns accuracy per batch
    """

    class_preds = nn.Softmax(dim=-1)(preds)
    class_preds = class_preds.max(-1)[1]
    correct = (class_preds == y).float() # convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

# we use the cross-entropy loss
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def train(model, train_iterator, valid_iterator, optimizier, criterion, N_EPOCHS=15):
    
    for epoch in range(N_EPOCHS):
    
        start_time = time.time()

        # To ensure the dropout is "turned on" while training
          # (good practice to include in your projects even if it is not used)
        model.train()
        
        epoch_loss = 0
        epoch_acc = 0
    
        # `batch` is a tuple of Tensors: (TEXT, LABEL)
        for batch in train_iterator:
                        
            # Zero the gradients
            optimizer.zero_grad()

            text = batch.text
            labels = batch.label
            # shape(text) = [T, B]
            # shape(label) = [B]
            
            # We reshape text to [B, T]. 
            # This is purely so we can think about the shapes of the Tensors more consistently
            text = text.T
            
            predictions = model(text)
            
            # compute the loss
            loss = criterion(predictions, labels)
        
            # compute training accuracy
            acc = accuracy(predictions, labels)
              
            # calculate the gradient of each parameter
            loss.backward()
        
            # update the parameters using the gradients and optimizer algorithm 
            optimizer.step()
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        average_epoch_loss = epoch_loss / len(train_iterator)
        average_epoch_acc = epoch_acc / len(train_iterator)
        
        end_time = time.time()
        
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
        average_epoch_valid_loss, average_epoch_valid_acc = evaluate(model, valid_iterator, criterion)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {average_epoch_loss:.3f} | Train Acc: {average_epoch_acc*100:.2f}%')
        print(f'\t Val. Loss: {average_epoch_valid_loss:.3f} |  Val. Acc: {average_epoch_valid_acc*100:.2f}%')
        

In [ ]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    # Turn on evaluate mode.
      # De-activates dropout and batch normalization
    model.eval()

    # We do not compute gradients within this block, i.e. no training
    # https://discuss.pytorch.org/t/model-eval-vs-with-torch-no-grad/19615/2
    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            labels = batch.label

            text = text.T
            
            predictions = model(text)
            loss = criterion(predictions, labels)
            acc = accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
train(model, train_iterator, valid_iterator, optimizer, criterion)

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## BiDirectional RNNs

In problems where all timesteps of the input sequence are available, bidirectional RNNs train two instead of one RNNs on the input sequence. The first on the input sequence as-is and the second on a reversed copy of the input sequence. Outputs at the same step are then usually concatenated. This can provide additional useful context to the model.

Q: Why is a bi-directional RNN is better than single-direction ?

Imagine that you see only the left context: "We went to ..." This context is very general and a lot of different words can continue: nouns (London, work, cinema, doctor), verbs (join, support), etc. When we both left and right contexts the word "sleep" is becoming evident: "We went to ... early but still could not wake up on time."


![bi_rnn_classification](images/bi_rnn_classification.png)

In [ ]:
BIDIRECTIONAL = True

bidirectional_model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [ ]:
train(bidirectional_model, train_iterator, valid_iterator, optimizer, criterion)

In [ ]:
test_loss, test_acc = evaluate(bidirectional_model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# RNNs for Language Modelling

Language model is required to represent the text to a form understandable from the machine point of view. Language Modelling (LM) is at the core of Natural Language Processing (NLP). Base of all the NLP tasks: Machine Translation, Spell Correction, Speech Recognition, Summarization, Question Answering, Sentiment analysis etc.  

Language is a sequence of letters, words, sentences, paragraphs, etc. These units are not independent. When we comprehend and produce spoken language, we are processing continuous input streams of indefinite
length. And even when dealing with written text we normally read it sequentially. Thus, RNNs is a perfect fit to model language data because with RNNs we can represent language sequence of any length into a fixed-sized vector.
 
Q: What is the difference between word embeddings and language modelling?

The main difference that word embeddings do not take into account word order. Language models take word order into account. The word order is important. If you do not take the word order into account the representation of the following sentences will be the same: "It was really not good, on the opposite quite bad." and "It was really not bad, on the opposite quite good." However, the meaning of those two sentences is different.

![rnn_lm](images/rnn_lm.png)


You may have heard about BERT. BERT is a general-purpose pre-trained language model. It is pre-trained using a lot of language data from Internet to create a better "grasp" of language. It is bidirectional. This means a deeper sense of language context and flow compared to the single-direction language models. You can download it and fine-tune for your NLP problem. 

In [ ]:
# With TorchText Field we define how our data will be processed
TEXT = data.Field(tokenize = 'spacy', init_token = '<sos>')

# We will be using the WikiText-2 corpus, which is a popular LM dataset.
# The WikiText language modeling dataset is a collection of texts extracted 
  # from good and featured articles on Wikipedia.
# It contains about 2 million words 
train_data, valid_data, test_data = datasets.WikiText2.splits(TEXT)

# Data stats
print('train.fields', train_data.fields)
print('len(train)', len(train_data))

# Build a vocabulary out of tokens available from the pre-trained embeddings list and the vocabulary of labels
TEXT.build_vocab(train_data, vectors="glove.6B.100d")
print('Text Vocabulary Length', len(TEXT.vocab))

BATCH_SIZE = 64

# place the tensors on the GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

The application of the Backpropagation training algorithm to RNNs applied to sequence data. Each timestep of the unrolled recurrent neural network may be seen as an additional layer given the order dependence of the problem and the internal state from the previous timestep is taken as an input on the subsequent timestep.


In [ ]:
# BPTTIterator (Backpropagation Through Time Iterator)
# divides the corpus into batches of [batch size, bptt_len/sequence length]

train_iterator, valid_iterator, test_iterator = data.BPTTIterator.splits(
            (train_data, valid_data, test_data), 
                batch_size = BATCH_SIZE, bptt_len=30,
                device = device, repeat=False)

for batch in train_iterator:
    demo_batch = batch
    break
    
print(demo_batch)

print()

# Note that the first dimension is the sequence, and the next is the batch.
  # We can reshape this to [batch size, sentence length] as we did earlier with a transpose.
# The target is the original text offset by 1
print("Demo batch `text`:\n", demo_batch.text[:5, :3])
print("Demo batch `target`:\n", demo_batch.target[:5, :3])

In [ ]:
class RNN(nn.Module):
    
    # variant is a flag which is either: "rnn", "lstm", "manual_lstm"
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout, pad_idx, variant):
        
        super().__init__()
        
        self.variant = variant
        
        self.embedding = nn.Embedding.from_pretrained(TEXT.vocab.vectors)

        # UNIDIRECTIONAL RNN layer: For LM modelling we do not see/have access to the right context
        
        if variant == "rnn":
            self.rnn = nn.RNN(embedding_dim, 
                               hidden_dim, 
                               batch_first=True)
        elif variant == "lstm":
            self.rnn = nn.LSTM(embedding_dim, 
                               hidden_dim, 
                               batch_first=True)
        elif variant == "manual_lstm":
            self.rnn = Manual_LSTM(embedding_dim, hidden_dim)
        else:
            raise ValueError("Expected `variant` to be one of 'rnn', 'lstm', or 'manual_lstm'")
            
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

       
    def forward(self, text, prev_hidden):
         
        # shape(text) = [B, T]
        
        # If vanilla RNN:
            # shape(prev_hidden) = [1, B, D] where 1 = num_layers*num_directions
        # If LSTM:
            # prev_hidden is a tuple of previous hidden states and cell states: (ALL_HIDDEN_STATES, ALL_CELL_STATES)
            # shape(ALL_HIDDEN_STATES)=shape(ALL_CELL_STATES) = [1, B, D] where 1 = num_layers*num_directions
            
        embedded = self.dropout(self.embedding(text))
        # shape(embedded) = [B, T, E]
        
        all_hidden, last_hidden = self.rnn(embedded, prev_hidden)        
        # shape(all_hidden) = [B, T, D]
        # shape(last_hidden) = [num layers, B, T]
        
        # Take all hidden states to produce an output word per time step
        logits = self.fc(self.dropout(all_hidden))
        # shape(logits) = [B, O]
            
        return logits, last_hidden

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(TEXT.vocab)
DROPOUT = 0.5
# get our pad token index from the vocabulary
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [ ]:
rnn_model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            DROPOUT, 
            PAD_IDX,
            variant="rnn")

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(rnn_model.parameters())

# we use the cross-entropy loss
criterion = nn.CrossEntropyLoss()

rnn_model = rnn_model.to(device)
criterion = criterion.to(device)

We need to detach the hidden state or else the model will try to backpropagate to the beginning of the dataset, requiring a lot of memory

In [ ]:
def save_hidden(hidden):
  """Wraps hidden states in new Tensors, to declare it not to need gradients. So that the initial hidden state for this batch is constant and doesn’t depend on anything."""

  if isinstance(hidden, torch.Tensor):
    return hidden.detach()
  else:
    return tuple(save_hidden(v) for v in hidden)


## Evaluation of Language Models

Language is very difficult to evaluate since there is no single gold truth: one meaning could be expressed in many valid ways.


### Human Evaluation

Human evaluation is costly, slow and subjective but reliable. Human evaluation of a language model may involve how a hypothesis satisfies the grammatical and lexical norms of a language.


### Perplexity

Does it prefer real (=frequently observed) sentences to ‘ungrammatical/gibberish’ (or rarely observed) ones? 
Remember that entropy is the average number of bits to encode the information contained in a random variable, so the exponentiation of the entropy (perplexity, $e^{H}$) should be the total amount of all possible information, or more precisely, the weighted average number of choices a random variable has. We evaluate our prediction Q by testing against samples drawn from P: $PPL = e^{H(P,Q)}$.

Measure perplexity on an unseen (test) corpus, generally we compare a range of models using this score. The best LM is the one that generates the lowest perplexity on the test corpus.


In [ ]:
def perplexity(loss_per_token):
    return math.exp(loss_per_token)

In [ ]:
def train(model, train_iterator, valid_iterator, optimizer, criterion, N_EPOCHS=10, is_lstm=False, force_stop=False):
    
    for epoch in range(N_EPOCHS):
    
        start_time = time.time()
        
        model.train()

        epoch_loss = 0
        epoch_items = 0
        
        # The `1` is the number of layers * number of directions.
        # i.e. we have 1 layer and we are moving in 1 direction
        # More info: https://pytorch.org/docs/stable/nn.html#rnn
        prev_hidden = torch.zeros(1, BATCH_SIZE, HIDDEN_DIM, device=device)
        if is_lstm:
            prev_ht = torch.zeros(1, BATCH_SIZE, HIDDEN_DIM, device=device)
            prev_ct = torch.zeros(1, BATCH_SIZE, HIDDEN_DIM, device=device)
            prev_hidden = (prev_ht, prev_ct)

        
        # `batch` is a tuple of Tensors: (TEXT, TARGET)
        for i, batch in enumerate(train_iterator):
            
            if force_stop:
                print("Currently processing train batch {} of {}".format(i, len(train_iterator)))
                if i % 7 == 0 and i != 0:
                    break
            
            # Zero the gradients
            optimizer.zero_grad()

            text = batch.text
            targets = batch.target
            # shape(text) = [T, B]
            # shape(target) = [T, B]
            
            # We reshape text and target to [B, T]. 
            text = text.T
            targets = targets.T
            # shape(text) = [B, T]
            # shape(target) = [B, T]
            
            # Starting each batch, we detach the hidden state from how it was previously produced.
            # Otherwise the model would backpropagate all the way to beginning of the dataset.
            prev_hidden = save_hidden(prev_hidden)
            
            ## Run the model
            
            
            # Compute the loss
            # We reshape inputs to eliminate batching
            loss = criterion(logits.view(-1, OUTPUT_DIM), targets.reshape(-1))
        
            ## Backprop and perform gradient descent
            
            epoch_loss += loss.detach().sum()
            epoch_items += loss.numel()
        
        # We compute loss per token for an epoch
        train_loss_per_token = epoch_loss / epoch_items
        # We compute perplexity
        train_ppl = perplexity(train_loss_per_token)

        end_time = time.time()        
        
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
        valid_loss_per_token, valid_ppl = evaluate(model, 
                                                   valid_iterator, 
                                                   criterion,
                                                   is_lstm=is_lstm,
                                                   force_stop=force_stop)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss_per_token:.3f} | Train Perplexity: {train_ppl:.3f}')
        print(f'\t Val. Loss: {valid_loss_per_token:.3f} |  Val. Perplexity: {valid_ppl:.3f}')
        
        if force_stop:
            break

In [ ]:
def evaluate(model, iterator, criterion, is_lstm=False, force_stop=False):
    
    model.eval()
    
    epoch_loss = 0
    epoch_items = 0
    
    # we initialise the first hidden state with zeros
    ## Initialise the previous hidden states of the RNNs
    ##prev_hidden = 
#     if is_lstm:
        ##prev_ht =
        ##prev_ct = 
        ##prev_hidden = 

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            
            if force_stop and i % 3 == 0 and i != 0:
                print("Currently processing valid batch {} of {}".format(i, len(train_iterator)))
                break

            text, target = batch.text, batch.target
            text, target = text.T, target.T
            logits, prev_hidden = model(text, prev_hidden)

            # compute the loss
            loss = criterion(logits.view(-1, OUTPUT_DIM), target.reshape(-1))

            prev_hidden = save_hidden(prev_hidden)

            epoch_loss += loss.detach().sum()
            epoch_items += loss.numel()

        loss_per_token = epoch_loss / epoch_items
        ppl = math.exp(loss_per_token)
            
        
    return loss_per_token, ppl

In [ ]:
train(rnn_model, train_iterator, valid_iterator, optimizer, criterion, force_stop=True)


## Long short-term memory architectures LSTMs vs. RNNs

**The vanishing gradient problem**. The gradient signal gets smaller and smaller as it backpropagates further. It is caused by the repeated use of the recurrent weight matrix in RNN. Gradient can be viewed as a measure of the effect of the past on the future. If the gradient becomes vanishingly small over longer distances we can not capture the dependency to the past correctly. For example: "A patient with a rare sarcoma of soft tissue on the left thigh was presented to the hospital yesterday." "was presented" depends on "a patient", but they are separated by 11 words!

![full_lstm](images/lstm_full.png)

The key to LSTMs is the cell state $c_t$. It runs straight down the entire chain and allow the information to just flow along it unchanged. LSTM has two "hidden states": $c_t$  and $h_t$ . You can think of $c_t$  is the "internal" hidden state that retains important information for longer timesteps, whereas $h_t$ is the "external" hidden state that exposes that information to the outside world.

The LSTM does have the ability to remove or add information to the cell state. Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation.  An LSTM has three of these gates.

Forget gate decides what information we’re going to throw away from the cell state. 

$f_t = \sigma(W_{if}x_t + W_{hf}h_{t-1}+b_f)$

$\sigma$ squashes input values between 0 and 1, describing how much of each component should be let through. Zero means "let nothing through", while a value of one means "let everything through".

![lstm_ft](images/lstm_ft.png)

Input gate decides what new information we are going to store in the cell state. 

$i_t = \sigma(W_{ii}x_t + W_{hi}h_{t-1}+b_i)$

Next, a tanh layer creates a vector of new candidate values, $g_t$, that could be added to the state. tanh squashes the output values to be between −1 and 1. 

$g_t = tanh(W_{ig}x_t + W_{hg}h_{t-1}+b_g)$ (this equation equal to vanilla RNN if we remove gates)

The next step combines these two to create an update to the state. Pointwise multiplication operation (*) decides on the parts we output.

$c_t = f_t * c_{t-1} + i_t * g_t$

![lstm_it_cand](images/lstm_it_cand.png)

Finally, the output gate decides how much information goes to the output:

$o_t = \sigma(W_{io}x_t + W_{ho}h_{t-1}+b_o)$

$h_t = o_t * tanh(c_t)$

![lstm_ot](images/lstm_ot.png)


Q: How does this help with the vanishing gradient problem ?

Whereas the RNN computes the new hidden state from scratch based on the previous hidden state and the input, the LSTM computes the new hidden state by choosing what to add to the current state. This allows skipping multiplicative gradient paths. Multiple additive paths are hard to converge to ~0.

Q: How are language models trained ?

Very often the so-called "teacher forcing" is used. It works by using the actual ground true outputs from the training dataset at the current time step t as input in the next time step t+1, rather than the output generated by the network. This makes learning faster and the model more stable. The model is not going to get punished for every subsequent word it generates. 

In [ ]:
class Manual_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.hidden_size = hidden_size

        self.forget_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Sigmoid()
        )
        ## Code up the input gate
        ## Code up the candidate gate
        ## Code up the output gate
        
        
        self.input_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Sigmoid()
        )

        self.candidate_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Tanh()
        )

        self.output_gate = nn.Sequential(
            nn.Linear(hidden_size+input_size, hidden_size),
            nn.Sigmoid()
        )

    def forward(self, x, prev_hidden):

        # shape(x) = [B, T, input_size]
        # shape(prev_hidden) = ([1, B, hidden_size], [1, B, hidden_size]) where 1 = num_layers * num_directions

        batch_size, sequence_length, _ = x.size()
        
        # At t=0, h_t and c_t will be initialized to a vector of 0s
        h_t = prev_hidden[0].squeeze(0)
        c_t = prev_hidden[1].squeeze(0)

        hidden_states = torch.zeros(batch_size, sequence_length, self.hidden_size).to(device)

        for t in range(sequence_length):

            # shape(x_t) = [B, input_size]
            x_t = x[:, t, :]

            # shape(concat_h_x) = [B, hidden_size+input_size]
            concat_h_x = torch.cat((h_t, x_t), dim=-1)

            # shape(f_t) = [B, hidden_size]
            f_t = self.forget_gate(concat_h_x)

            # shape(c_prime_t) = [B, hidden_size]
            c_prime_t = c_t * f_t

            # shape(i_t) = [B, hidden_size]
            # shape(cand_t) = [B, hidden_size]
            i_t = self.input_gate(concat_h_x)
            cand_t = self.candidate_gate(concat_h_x)

            # shape(c_t) = [B, hidden_size]
            c_t = c_prime_t + (i_t * cand_t)

            # shape(o_t) = [B, hidden_size]
            # shape(h_t) = [B, hidden_size]
            o_t = self.output_gate(concat_h_x)
            h_t = o_t * torch.tanh(c_t)

            hidden_states[:, t, :] = h_t

        h_t, c_t = h_t.unsqueeze(0), c_t.unsqueeze(0)
        return hidden_states, (h_t, c_t)

In [ ]:
manual_lstm = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,
            DROPOUT, 
            PAD_IDX,
            variant="manual_lstm")

lstm = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM,
            DROPOUT, 
            PAD_IDX,
            variant="lstm")

In [ ]:
optimizer = optim.Adam(manual_lstm.parameters())
train(manual_lstm, train_iterator, valid_iterator, optimizer, criterion, is_lstm=True, force_stop=True)

In [ ]:
optimizer = optim.Adam(lstm.parameters())
train(lstm, train_iterator, valid_iterator, optimizer, criterion, is_lstm=True, force_stop=True)

How did the RNN, Manual LSTM and LSTM fare up against each other?

![vanilla_rnn_lm](images/vanilla_rnn_lm.png)
![manual_lstm_lm](images/manual_lstm_lm.png)
![lstm_lm](images/lstm_lm.png)

## LSTMs vs. GRUs

Gated Recurrent Unit (GRU) combines the forget and input gates into a single "update gate" (z). So we have only two gates: update and reset. It also merges the cell state and hidden state, and makes some other changes. The resulting model is simpler than standard LSTM models. Candidate state $g_t$ is able to suppress $h_t$. The final state is a convex combination: of the $g_t$ and $h_{t-1}$ with coefficients of $(1 - z_t)$ and $z_t$ respectively.

$r_t = \sigma(W_{ir}x_t + W_{hr}h_{t-1}+b_r)$

$z_t = \sigma(W_{iz}x_t + W_{hz}h_{t-1}+b_z)$

$g_t = tanh(W_{ig}x_t + r_t * (W_{hg}h_{t-1}+b_g))$

$h_t = (1 - z_t)* g_t + z_t * h_{t-1}$

In [ ]:
# self.gru = nn.GRU(...)